# Aggregate BACI

Products in the BACI dataset are at the 6-digit HS level. Depending on the use case, it may be preferable to work with an aggregated version of the dataset. This notebook contains scripts for aggregating at the 4-digit, the 2-digit, and the national levels.

Because the column `q` (weight of trade flows in metric tons) has a substantial amount of missing values, it is dropped here.

Run `save-to-parquet.ipynb` before running this notebook. Set `hs` and `release` below to correpond to the selected version of BACI.

In [1]:
hs = 'HS17'             # Change this!
release = '202301'      # Change this!

filename = f'BACI_{hs}_V{release}'

In [2]:
import duckdb

## Country level

In [3]:
duckdb.sql(
    f"""
    COPY (
        SELECT t, i, j, SUM(v) AS v
        FROM 'final/{filename}.parquet'
        GROUP BY t, i, j
        ORDER BY t
    ) TO 'final/{filename}-country.parquet'
    """
)

In [4]:
duckdb.sql(f"SELECT * FROM 'final/{filename}-country.parquet' LIMIT 10").df()

,t,i,j,v
0,2017,516,36,9114.438
1,2017,516,68,9.775
2,2017,516,116,0.084
3,2017,516,152,203.483
4,2017,516,262,2.093
5,2017,516,178,91.241
6,2017,516,218,199.916
7,2017,516,304,1.670
8,2017,516,484,659.135
9,2017,516,586,2509.611


## 2-digit level

In [5]:
duckdb.sql(
    f"""
    COPY (
        SELECT t, i, j, k2, SUM(v) AS v
        FROM (
            SELECT t, i, j, SUBSTRING(k, -6, 2) AS k2, v
            FROM 'final/{filename}.parquet'
        )
        GROUP BY t, i, j, k2
        ORDER BY t
    ) TO 'final/{filename}-2digit.parquet'
    """
)

In [6]:
duckdb.sql(f"SELECT * FROM 'final/{filename}-2digit.parquet' LIMIT 10").df()

,t,i,j,k2,v
0,2017,56,320,20,808.253
1,2017,56,320,22,310.911
2,2017,56,320,24,24.408
3,2017,56,320,48,2975.069
4,2017,56,320,66,0.177
5,2017,56,320,92,4.011
6,2017,56,324,16,1.629
7,2017,56,324,74,106.642
8,2017,56,324,79,0.069
9,2017,56,324,86,193.680


## 4-digit level

In [7]:
duckdb.sql(
    f"""
    COPY (
        SELECT t, i, j, k4, sum(v) AS v
        FROM (
            SELECT t, i, j, substring(k, -6, 4) AS k4, v
            FROM 'final/{filename}.parquet'
        )
        GROUP BY t, i, j, k4
        ORDER BY t
    ) TO 'final/{filename}-4digit.parquet'
    """
)

In [8]:
duckdb.sql(f"SELECT * FROM 'final/{filename}-4digit.parquet' LIMIT 10").df()

,t,i,j,k4,v
0,2017,36,704,8533,2.463
1,2017,36,704,8534,13.026
2,2017,36,704,9007,116.628
3,2017,36,704,9209,19.814
4,2017,36,705,1102,3.982
5,2017,36,705,1108,8.343
6,2017,36,705,2924,0.114
7,2017,36,705,3307,38.995
8,2017,36,705,3925,2.037
9,2017,36,705,6109,1.674
